In [30]:
import pandas as pd

from keras.models import Sequential
from keras.layers import LSTM, Dense

In [31]:
flights = pd.read_csv("Dataset_Post_Engineer.csv", sep=",")
flights.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,...,Total_Quantity,DayoftheWeek,WeekoftheYear,Month,Week_Month_Label,DUR_INTERVAL,Time_Interval,DayoftheWeek_E,Time_Interval_E,Week_Month_Label_E
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,...,42.0,Thursday,42,10,Week 42 (Oct),2.0,06:00-12:00,4.0,1.0,42.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,186.0,...,15.0,Monday,27,7,Week 27 (Jul),3.0,00:00-06:00,1.0,0.0,22.0
2,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,...,0.0,Monday,26,6,Week 26 (Jun),0.0,12:00-18:00,1.0,2.0,21.0
3,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,...,22.0,Friday,6,2,Week 6 (Feb),1.0,06:00-12:00,0.0,1.0,57.0
4,3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,...,2.0,Thursday,36,9,Week 36 (Sep),0.0,12:00-18:00,4.0,2.0,34.0


In [32]:
categories = ['Quantity_Alimentos Charter', 'Quantity_Bebidas Calientes', 'Quantity_Botanas', 'Quantity_Galletas',
              'Quantity_Lacteos', 'Quantity_Licores', 'Quantity_Perecederos', 'Quantity_Refrescos', 'Quantity_Sopas']

temp = flights[["Time_Interval_E", "DUR_HR", "DepartureStation_Encoded", "ArrivalStation_Encoded", "Capacity", "Bookings", "WeekoftheYear"] + categories]

sorted_flights = temp.sort_values("WeekoftheYear")
sorted_flights

,Time_Interval_E,DUR_HR,DepartureStation_Encoded,ArrivalStation_Encoded,Capacity,Bookings,WeekoftheYear,Quantity_Alimentos Charter,Quantity_Bebidas Calientes,Quantity_Botanas,Quantity_Galletas,Quantity_Lacteos,Quantity_Licores,Quantity_Perecederos,Quantity_Refrescos,Quantity_Sopas
116210,1.0,1.5000,10.0,37.0,186,122.0,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0
117714,1.0,1.5000,16.0,37.0,240,125.0,1,0.0,1.0,2.0,4.0,1.0,0.0,0.0,3.0,0.0
112795,2.0,1.7500,15.0,34.0,240,140.0,1,0.0,3.0,37.0,4.0,2.0,18.0,11.0,42.0,6.0
12673,1.0,2.6667,38.0,7.0,186,117.0,1,0.0,3.0,3.0,3.0,1.0,0.0,2.0,8.0,0.0
116013,1.0,0.8333,7.0,37.0,186,66.0,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18013,2.0,3.3333,18.0,7.0,186,71.0,52,0.0,0.0,22.0,0.0,0.0,6.0,2.0,29.0,2.0
73932,3.0,1.5833,15.0,18.0,186,136.0,52,0.0,0.0,1.0,0.0,0.0,2.0,0.0,6.0,2.0
73940,3.0,2.1667,16.0,18.0,240,92.0,52,0.0,0.0,8.0,1.0,3.0,3.0,0.0,18.0,1.0
7548,1.0,1.6667,15.0,4.0,240,131.0,52,0.0,1.0,1.0,4.0,0.0,0.0,0.0,4.0,0.0


In [33]:
super_dataframe = pd.DataFrame()

unique_salidas = sorted_flights["DepartureStation_Encoded"].unique()

for origen in unique_salidas:
    temp = sorted_flights[sorted_flights["DepartureStation_Encoded"] == origen]
    
    unique_destinos = temp["ArrivalStation_Encoded"].unique()

    for destino in unique_destinos:
        temp_destino = temp[temp["ArrivalStation_Encoded"] == destino]
        temp_destino = temp_destino.drop(["DepartureStation_Encoded", "ArrivalStation_Encoded"], axis=1)

        unique_horarios = temp_destino["Time_Interval_E"].unique()

        for horario in unique_horarios:
            temp_horario = temp_destino[temp_destino["Time_Interval_E"] == horario]
            temp_horario = temp_horario.drop(["Time_Interval_E"], axis=1)

            temp_horario = temp_horario.groupby("WeekoftheYear").mean()

            temp_horario["DepartureStation_Encoded"] = origen
            temp_horario["ArrivalStation_Encoded"] = destino
            temp_horario["Time_Interval_E"] = horario

            temp_horario.reset_index(names=["WeekoftheYear"], inplace=True)

            key = str(origen) +"-" + str(destino) + "-" + str(horario)

            temp_horario["id"] = key

            super_dataframe = pd.concat([super_dataframe, temp_horario], axis=0, ignore_index=True)


#            print(temp_horario)

super_dataframe

,WeekoftheYear,DUR_HR,Capacity,Bookings,Quantity_Alimentos Charter,Quantity_Bebidas Calientes,Quantity_Botanas,Quantity_Galletas,Quantity_Lacteos,Quantity_Licores,Quantity_Perecederos,Quantity_Refrescos,Quantity_Sopas,DepartureStation_Encoded,ArrivalStation_Encoded,Time_Interval_E,id
0,1,1.5000,186.000000,105.571429,0.0,1.571429,5.571429,1.000000,0.714286,0.285714,2.714286,6.285714,0.142857,10.0,37.0,1.0,10.0-37.0-1.0
1,2,1.5000,185.142857,113.571429,0.0,0.428571,0.714286,0.571429,0.000000,0.142857,0.571429,2.000000,0.000000,10.0,37.0,1.0,10.0-37.0-1.0
2,3,1.5000,185.142857,125.285714,0.0,1.285714,1.571429,0.285714,0.142857,0.285714,0.857143,2.714286,0.142857,10.0,37.0,1.0,10.0-37.0-1.0
3,4,1.5000,186.000000,116.285714,0.0,1.000000,0.857143,0.285714,0.000000,0.285714,0.285714,1.142857,0.000000,10.0,37.0,1.0,10.0-37.0-1.0
4,5,1.5000,186.000000,122.000000,0.0,0.571429,0.000000,0.714286,0.285714,0.000000,0.142857,1.285714,0.000000,10.0,37.0,1.0,10.0-37.0-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20676,51,1.9167,240.000000,69.666667,0.0,5.000000,45.000000,7.666667,0.333333,10.333333,19.666667,46.333333,16.666667,40.0,34.0,2.0,40.0-34.0-2.0
20677,52,1.9167,240.000000,82.333333,0.0,6.000000,58.666667,12.333333,2.666667,10.666667,19.666667,63.000000,12.333333,40.0,34.0,2.0,40.0-34.0-2.0
20678,50,1.9167,186.000000,91.000000,0.0,5.000000,39.000000,4.000000,1.000000,11.000000,14.000000,27.000000,12.000000,40.0,10.0,1.0,40.0-10.0-1.0
20679,51,1.9167,184.000000,89.000000,0.0,5.000000,30.666667,5.333333,1.666667,6.666667,12.666667,33.333333,8.000000,40.0,10.0,1.0,40.0-10.0-1.0


In [34]:
for i in super_dataframe["id"].unique():
    
    temp = super_dataframe[super_dataframe["id"] == i]
    times = temp["WeekoftheYear"]

    start_index = temp.index[0]
    end_index = temp.index[-1]

    prev = float("inf")

    cont = 1

    for j, time in enumerate(times):
        j = j + start_index

        if time - prev > 2:
            super_dataframe.loc[j:end_index, "id"] = super_dataframe.loc[j:end_index, "id"] + str(cont)
            cont += 1

        prev = time



In [35]:
ids_to_remove = []
for i in super_dataframe["id"].unique():
    if len(super_dataframe[super_dataframe["id"] == i]["WeekoftheYear"]) < 4:
        ids_to_remove.append(i)
ids_to_remove

['10.0-37.0-2.0',
 '10.0-37.0-2.012',
 '10.0-37.0-2.0123',
 '10.0-29.0-3.01',
 '10.0-8.0-2.01',
 '10.0-8.0-1.01',
 '10.0-4.0-0.0',
 '10.0-15.0-0.0',
 '10.0-34.0-0.0',
 '10.0-18.0-0.0',
 '10.0-38.0-2.01',
 '10.0-38.0-3.01',
 '10.0-24.0-3.0',
 '10.0-28.0-3.0',
 '10.0-28.0-3.01',
 '10.0-25.0-2.0',
 '10.0-7.0-0.0',
 '10.0-13.0-1.01',
 '10.0-11.0-2.01',
 '10.0-22.0-2.0',
 '10.0-40.0-1.0',
 '16.0-37.0-2.01',
 '16.0-37.0-2.012',
 '16.0-38.0-3.0',
 '16.0-38.0-3.01',
 '16.0-38.0-1.0',
 '16.0-38.0-1.01',
 '16.0-38.0-2.012',
 '16.0-18.0-2.0',
 '16.0-18.0-0.0',
 '16.0-35.0-1.01',
 '16.0-35.0-2.0',
 '16.0-35.0-3.0',
 '16.0-35.0-3.01',
 '16.0-15.0-0.0',
 '16.0-15.0-0.012',
 '16.0-27.0-3.0',
 '16.0-27.0-3.01',
 '16.0-27.0-1.0',
 '16.0-22.0-3.0',
 '15.0-5.0-2.01',
 '15.0-16.0-0.0',
 '15.0-10.0-0.0',
 '15.0-10.0-0.01',
 '15.0-38.0-2.0',
 '15.0-38.0-0.0',
 '15.0-38.0-0.01',
 '15.0-31.0-1.0',
 '15.0-25.0-3.0',
 '15.0-25.0-0.0',
 '15.0-8.0-3.0',
 '15.0-26.0-0.0',
 '15.0-33.0-2.0',
 '15.0-33.0-2.01',
 '15.

In [36]:
super_dataframe = super_dataframe[~super_dataframe["id"].isin(ids_to_remove)]
super_dataframe

,WeekoftheYear,DUR_HR,Capacity,Bookings,Quantity_Alimentos Charter,Quantity_Bebidas Calientes,Quantity_Botanas,Quantity_Galletas,Quantity_Lacteos,Quantity_Licores,Quantity_Perecederos,Quantity_Refrescos,Quantity_Sopas,DepartureStation_Encoded,ArrivalStation_Encoded,Time_Interval_E,id
0,1,1.50,186.000000,105.571429,0.0,1.571429,5.571429,1.000000,0.714286,0.285714,2.714286,6.285714,0.142857,10.0,37.0,1.0,10.0-37.0-1.0
1,2,1.50,185.142857,113.571429,0.0,0.428571,0.714286,0.571429,0.000000,0.142857,0.571429,2.000000,0.000000,10.0,37.0,1.0,10.0-37.0-1.0
2,3,1.50,185.142857,125.285714,0.0,1.285714,1.571429,0.285714,0.142857,0.285714,0.857143,2.714286,0.142857,10.0,37.0,1.0,10.0-37.0-1.0
3,4,1.50,186.000000,116.285714,0.0,1.000000,0.857143,0.285714,0.000000,0.285714,0.285714,1.142857,0.000000,10.0,37.0,1.0,10.0-37.0-1.0
4,5,1.50,186.000000,122.000000,0.0,0.571429,0.000000,0.714286,0.285714,0.000000,0.142857,1.285714,0.000000,10.0,37.0,1.0,10.0-37.0-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20668,52,1.50,192.000000,98.142857,0.0,2.000000,27.571429,3.000000,1.142857,10.000000,9.714286,28.142857,9.571429,40.0,15.0,2.0,40.0-15.0-2.0
20669,48,1.25,186.000000,121.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.0,22.0,2.0,40.0-22.0-2.0
20670,50,1.25,186.000000,113.666667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.0,22.0,2.0,40.0-22.0-2.0
20671,51,1.25,184.285714,96.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.0,22.0,2.0,40.0-22.0-2.0


In [37]:
import math
unicos = super_dataframe["id"].unique()

sizes = []

window = 4

for i in unicos:
    series = super_dataframe[super_dataframe["id"] == i]

    if series.empty:
        continue

    weeks = series["WeekoftheYear"]

    start_index = series.index[0]
    end_index = start_index + window - 1

    num_sequences = math.floor(len(weeks) / window)

    cont = "a"

    for i in range(num_sequences):
        
        super_dataframe.loc[start_index: end_index, "id"] = super_dataframe.loc[start_index: end_index, "id"] + cont
        
        start_index = end_index + 1
        end_index = end_index + window
        
        cont += "a"
        #print((i+1)*window + start_index - start_index + i*window)
    

In [38]:
super_dataframe

,WeekoftheYear,DUR_HR,Capacity,Bookings,Quantity_Alimentos Charter,Quantity_Bebidas Calientes,Quantity_Botanas,Quantity_Galletas,Quantity_Lacteos,Quantity_Licores,Quantity_Perecederos,Quantity_Refrescos,Quantity_Sopas,DepartureStation_Encoded,ArrivalStation_Encoded,Time_Interval_E,id
0,1,1.50,186.000000,105.571429,0.0,1.571429,5.571429,1.000000,0.714286,0.285714,2.714286,6.285714,0.142857,10.0,37.0,1.0,10.0-37.0-1.0a
1,2,1.50,185.142857,113.571429,0.0,0.428571,0.714286,0.571429,0.000000,0.142857,0.571429,2.000000,0.000000,10.0,37.0,1.0,10.0-37.0-1.0a
2,3,1.50,185.142857,125.285714,0.0,1.285714,1.571429,0.285714,0.142857,0.285714,0.857143,2.714286,0.142857,10.0,37.0,1.0,10.0-37.0-1.0a
3,4,1.50,186.000000,116.285714,0.0,1.000000,0.857143,0.285714,0.000000,0.285714,0.285714,1.142857,0.000000,10.0,37.0,1.0,10.0-37.0-1.0a
4,5,1.50,186.000000,122.000000,0.0,0.571429,0.000000,0.714286,0.285714,0.000000,0.142857,1.285714,0.000000,10.0,37.0,1.0,10.0-37.0-1.0aa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20668,52,1.50,192.000000,98.142857,0.0,2.000000,27.571429,3.000000,1.142857,10.000000,9.714286,28.142857,9.571429,40.0,15.0,2.0,40.0-15.0-2.0
20669,48,1.25,186.000000,121.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.0,22.0,2.0,40.0-22.0-2.0a
20670,50,1.25,186.000000,113.666667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.0,22.0,2.0,40.0-22.0-2.0a
20671,51,1.25,184.285714,96.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.0,22.0,2.0,40.0-22.0-2.0a


In [39]:
sizes = []
for i in super_dataframe["id"].unique():
    sizes.append(len(super_dataframe[super_dataframe["id"] == i]["WeekoftheYear"]))
    #print(len(super_dataframe[super_dataframe["id"] == i]["WeekoftheYear"]))
set(sizes)

{1, 2, 3, 4}

In [40]:
ids_to_remove = []
for i in super_dataframe["id"].unique():
    if len(super_dataframe[super_dataframe["id"] == i]["WeekoftheYear"]) < 4:
        ids_to_remove.append(i)
ids_to_remove

['10.0-37.0-2.01',
 '10.0-37.0-3.0',
 '10.0-29.0-3.0',
 '10.0-8.0-2.0',
 '10.0-8.0-1.0',
 '10.0-4.0-2.0',
 '10.0-4.0-0.01',
 '10.0-38.0-1.0',
 '10.0-38.0-3.0',
 '10.0-16.0-2.0',
 '10.0-16.0-3.0',
 '10.0-16.0-3.01',
 '10.0-24.0-2.0',
 '10.0-24.0-2.01',
 '10.0-24.0-0.0',
 '10.0-28.0-2.0',
 '10.0-25.0-2.01',
 '10.0-25.0-3.0',
 '10.0-13.0-1.0',
 '10.0-13.0-2.0',
 '10.0-33.0-2.0',
 '10.0-33.0-2.01',
 '10.0-33.0-3.012',
 '10.0-11.0-2.0',
 '10.0-11.0-1.0',
 '10.0-11.0-3.0',
 '16.0-33.0-3.01',
 '16.0-33.0-1.0',
 '16.0-33.0-2.0',
 '16.0-38.0-3.012',
 '16.0-38.0-2.0',
 '16.0-38.0-2.0123',
 '16.0-2.0-1.0',
 '16.0-18.0-2.01',
 '16.0-18.0-2.012',
 '16.0-35.0-1.0',
 '16.0-35.0-2.01',
 '16.0-10.0-1.0',
 '16.0-10.0-1.01',
 '16.0-10.0-1.012',
 '16.0-10.0-2.0',
 '16.0-15.0-0.01',
 '16.0-24.0-2.0',
 '15.0-7.0-0.0',
 '15.0-5.0-2.0',
 '15.0-16.0-0.01',
 '15.0-38.0-1.0',
 '15.0-38.0-1.01',
 '15.0-25.0-3.01',
 '15.0-25.0-1.0',
 '15.0-29.0-3.0',
 '15.0-29.0-1.0',
 '15.0-21.0-2.0',
 '15.0-0.0-2.0',
 '15.0-4.0-

In [41]:
super_dataframe = super_dataframe[~super_dataframe["id"].isin(ids_to_remove)]
super_dataframe

,WeekoftheYear,DUR_HR,Capacity,Bookings,Quantity_Alimentos Charter,Quantity_Bebidas Calientes,Quantity_Botanas,Quantity_Galletas,Quantity_Lacteos,Quantity_Licores,Quantity_Perecederos,Quantity_Refrescos,Quantity_Sopas,DepartureStation_Encoded,ArrivalStation_Encoded,Time_Interval_E,id
0,1,1.50,186.000000,105.571429,0.0,1.571429,5.571429,1.000000,0.714286,0.285714,2.714286,6.285714,0.142857,10.0,37.0,1.0,10.0-37.0-1.0a
1,2,1.50,185.142857,113.571429,0.0,0.428571,0.714286,0.571429,0.000000,0.142857,0.571429,2.000000,0.000000,10.0,37.0,1.0,10.0-37.0-1.0a
2,3,1.50,185.142857,125.285714,0.0,1.285714,1.571429,0.285714,0.142857,0.285714,0.857143,2.714286,0.142857,10.0,37.0,1.0,10.0-37.0-1.0a
3,4,1.50,186.000000,116.285714,0.0,1.000000,0.857143,0.285714,0.000000,0.285714,0.285714,1.142857,0.000000,10.0,37.0,1.0,10.0-37.0-1.0a
4,5,1.50,186.000000,122.000000,0.0,0.571429,0.000000,0.714286,0.285714,0.000000,0.142857,1.285714,0.000000,10.0,37.0,1.0,10.0-37.0-1.0aa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20667,51,1.50,196.000000,111.142857,0.0,2.142857,20.571429,3.285714,0.571429,7.857143,5.714286,25.285714,8.571429,40.0,15.0,2.0,40.0-15.0-2.0a
20669,48,1.25,186.000000,121.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.0,22.0,2.0,40.0-22.0-2.0a
20670,50,1.25,186.000000,113.666667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.0,22.0,2.0,40.0-22.0-2.0a
20671,51,1.25,184.285714,96.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.0,22.0,2.0,40.0-22.0-2.0a


In [42]:
sizes = []
for i in super_dataframe["id"].unique():
    sizes.append(len(super_dataframe[super_dataframe["id"] == i]["WeekoftheYear"]))
    #print(len(super_dataframe[super_dataframe["id"] == i]["WeekoftheYear"]))
set(sizes)

{4}

### All elements are already of shape 4, we can train now...

In [43]:
super_dataframe

,WeekoftheYear,DUR_HR,Capacity,Bookings,Quantity_Alimentos Charter,Quantity_Bebidas Calientes,Quantity_Botanas,Quantity_Galletas,Quantity_Lacteos,Quantity_Licores,Quantity_Perecederos,Quantity_Refrescos,Quantity_Sopas,DepartureStation_Encoded,ArrivalStation_Encoded,Time_Interval_E,id
0,1,1.50,186.000000,105.571429,0.0,1.571429,5.571429,1.000000,0.714286,0.285714,2.714286,6.285714,0.142857,10.0,37.0,1.0,10.0-37.0-1.0a
1,2,1.50,185.142857,113.571429,0.0,0.428571,0.714286,0.571429,0.000000,0.142857,0.571429,2.000000,0.000000,10.0,37.0,1.0,10.0-37.0-1.0a
2,3,1.50,185.142857,125.285714,0.0,1.285714,1.571429,0.285714,0.142857,0.285714,0.857143,2.714286,0.142857,10.0,37.0,1.0,10.0-37.0-1.0a
3,4,1.50,186.000000,116.285714,0.0,1.000000,0.857143,0.285714,0.000000,0.285714,0.285714,1.142857,0.000000,10.0,37.0,1.0,10.0-37.0-1.0a
4,5,1.50,186.000000,122.000000,0.0,0.571429,0.000000,0.714286,0.285714,0.000000,0.142857,1.285714,0.000000,10.0,37.0,1.0,10.0-37.0-1.0aa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20667,51,1.50,196.000000,111.142857,0.0,2.142857,20.571429,3.285714,0.571429,7.857143,5.714286,25.285714,8.571429,40.0,15.0,2.0,40.0-15.0-2.0a
20669,48,1.25,186.000000,121.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.0,22.0,2.0,40.0-22.0-2.0a
20670,50,1.25,186.000000,113.666667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.0,22.0,2.0,40.0-22.0-2.0a
20671,51,1.25,184.285714,96.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.0,22.0,2.0,40.0-22.0-2.0a


In [44]:
X = super_dataframe[["Time_Interval_E", "DUR_HR", "DepartureStation_Encoded", "ArrivalStation_Encoded", "Capacity", "Bookings", "WeekoftheYear"]]

categories = ['Quantity_Alimentos Charter', 'Quantity_Bebidas Calientes', 'Quantity_Botanas', 'Quantity_Galletas',
              'Quantity_Lacteos', 'Quantity_Licores', 'Quantity_Perecederos', 'Quantity_Refrescos', 'Quantity_Sopas']

Y = super_dataframe[categories]

n_samples = len(super_dataframe["id"].unique())
n_timesteps = 4
n_features = X.shape[1]

output_dim = Y.shape[1]

def build_model(n_timestamps, n_features, output_dim):
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=(n_timestamps, n_features)))
    model.add(Dense(output_dim))
    model.compile(optimizer = 'adam', loss = 'mse')
    return model



In [45]:
X = X.to_numpy()
Y = Y.to_numpy()

n, m = X.shape
u = n // 4

n, v = Y.shape

transformed_X = X.reshape(u, 4, m)
transformed_Y = Y.reshape(u, 4, v)

In [46]:
transformed_Y.shape
#transformed_X.shape

(4842, 4, 9)

In [47]:
model = build_model(n_timesteps, n_features, output_dim)
model.fit(transformed_X, transformed_Y, epochs=40, batch_size=n_timesteps, validation_split=0.2)

Epoch 1/40
969/969 [==============================] - 9s 7ms/step - loss: 38.0007 - val_loss: 38.8761
Epoch 2/40
969/969 [==============================] - 6s 7ms/step - loss: 31.7257 - val_loss: 38.1549
Epoch 3/40
969/969 [==============================] - 6s 7ms/step - loss: 30.5175 - val_loss: 37.6189
Epoch 4/40
969/969 [==============================] - 6s 6ms/step - loss: 29.5868 - val_loss: 37.6500
Epoch 5/40
969/969 [==============================] - 6s 6ms/step - loss: 28.9081 - val_loss: 38.5937
Epoch 6/40
969/969 [==============================] - 6s 6ms/step - loss: 28.2014 - val_loss: 37.2668
Epoch 7/40
969/969 [==============================] - 8s 8ms/step - loss: 27.7476 - val_loss: 37.1044
Epoch 8/40
969/969 [==============================] - 8s 8ms/step - loss: 27.4843 - val_loss: 36.5548
Epoch 9/40
969/969 [==============================] - 8s 8ms/step - loss: 26.9752 - val_loss: 36.5291
Epoch 10/40
969/969 [==============================] - 8s 8ms/step - loss: 26.7241

In [23]:
transformed_X[0].shape

(4, 6)

In [48]:
model.evaluate(transformed_X, transformed_Y, batch_size=n_timesteps)

1211/1211 [==============================] - 4s 3ms/step - loss: 24.1262


24.126218795776367

In [21]:
import numpy as np
pred = model.predict(transformed_X[0:1]) 
pred = np.argmax(pred, axis = 1)[:5] 
#label = np.argmax(y_test,axis = 1)[:5]
pred 

1/1 [==============================] - 1s 505ms/step


array([[1, 1, 0, 1, 0, 0, 0, 0, 0]])

In [22]:
transformed_Y[0]

array([[0.        , 1.57142857, 5.57142857, 1.        , 0.71428571,
        0.28571429, 2.71428571, 6.28571429, 0.14285714],
       [0.        , 0.42857143, 0.71428571, 0.57142857, 0.        ,
        0.14285714, 0.57142857, 2.        , 0.        ],
       [0.        , 1.28571429, 1.57142857, 0.28571429, 0.14285714,
        0.28571429, 0.85714286, 2.71428571, 0.14285714],
       [0.        , 1.        , 0.85714286, 0.28571429, 0.        ,
        0.28571429, 0.28571429, 1.14285714, 0.        ]])

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=25b1adb0-2b47-474f-a7b7-c06d723a95f4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>